In [ ]:
import numpy as np 
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.ensemble import RandomForestRegressor 
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import xgboost as xgb
import optuna
import shap


pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)
InteractiveShell.ast_node_interactivity = "all"

# データ読み込み

In [ ]:
# パスの設定
input_path = '../input/home-credit-default-risk/'

application_train = pd.read_csv(input_path+'application_train.csv')
application_test = pd.read_csv(input_path+'application_test.csv')
bureau = pd.read_csv(input_path+'bureau.csv')
bureau_balance = pd.read_csv(input_path+'bureau_balance.csv')
previous_application = pd.read_csv(input_path+'previous_application.csv')
POS_CASH_balance = pd.read_csv(input_path+'POS_CASH_balance.csv')
installments_payments = pd.read_csv(input_path+'installments_payments.csv')
credit_card_balance = pd.read_csv(input_path+'credit_card_balance.csv')

# sample submissionデータの読み込み
sample_submission = pd.read_csv(input_path +'sample_submission.csv')

# データの確認

In [ ]:
# データ量の確認
print('Size of application_train data', application_train.shape)
print('Size of application_test data', application_test.shape)
print('Size of bureau data', bureau.shape)
print('Size of bureau_balance data', bureau_balance.shape)
print('Size of previous_application data', previous_application.shape)
print('Size of POS_CASH_balance data', POS_CASH_balance.shape)
print('Size of installments_payments data', installments_payments.shape)
print('Size of credit_card_balance data', credit_card_balance.shape)

In [ ]:
# データ確認
## train データ確認
print("\n======= train data =======\n")
application_train.shape
application_train.columns
application_train.info()
application_train.head(5)

## test データ確認
print("\n======= test data =======\n")
application_test.shape
application_test.columns
application_test.info()
application_test.head(5)

## bureau.csv データ確認
print("\n======= bureau data =======\n")
bureau.shape
bureau.columns
bureau.info()
bureau.head(5)

## bureau_balance.csv データ確認
print("\n======= bureau_balance data =======\n")
bureau_balance.shape
bureau_balance.columns
bureau_balance.info()
bureau_balance.head(5)

## previous_application.csv データ確認
print("\n======= previous_application data =======\n")
previous_application.shape
previous_application.columns
previous_application.info()
previous_application.head(5)

## POS_CASH_balance.csv データ確認
print("\n======= POS_CASH_balance data =======\n")
POS_CASH_balance.shape
POS_CASH_balance.columns
POS_CASH_balance.info()
POS_CASH_balance.head(5)

## installments_payments.csv データ確認
print("\n======= installments_payment data =======\n")
installments_payments.shape
installments_payments.columns
installments_payments.info()
installments_payments.head(5)

## credit_card_balance.csv データ確認
print("\n======= credit_card_balance data =======\n")
credit_card_balance.shape
credit_card_balance.columns
credit_card_balance.info()
credit_card_balance.head(5)

In [ ]:
# 基礎統計量を確認
## train 基礎統計量確認
print("\n======= train data =======\n")
application_train.describe().T

## test 基礎統計量確認
print("\n======= test data =======\n")
application_test.describe().T

## bureau.csv 基礎統計量確認
print("\n======= bureau data =======\n")
bureau.describe().T

## bureau_balance.csv 基礎統計量確認
print("\n======= bureau_balance data =======\n")
bureau_balance.describe().T

## previous_application.csv 基礎統計量確認
print("\n======= previous_application data =======\n")
previous_application.describe().T

## POS_CASH_balance.csv 基礎統計量確認
print("\n======= POS_CASH_balance data =======\n")
POS_CASH_balance.describe().T

## installments_payments.csv 基礎統計量確認
print("\n======= installments_payment data =======\n")
installments_payments.describe().T

## credit_card_balance.csv 基礎統計量確認
print("\n======= credit_card_balance data =======\n")
credit_card_balance.describe().T

# ランダムサンプリング
ランダムサンプリングを行った際にサンプル内のデフォルトしたか否かの割合が変わらないことを確認したい

In [ ]:
# trainのtarget=1となるサンプルの数を確認
print(application_train[application_train['TARGET']==1]['TARGET'].sum())

In [ ]:
# サンプル数が1/10倍になるようランダムサンプリング
_,application_train = train_test_split(application_train, test_size=0.1, stratify=application_train['TARGET'], random_state=42)
#application_train=application_train.sample(frac=0.1, random_state=2)

# サンプリング後のtrainのtarget=1となるサンプルの数の割合に変化がないか確認
print(application_train[application_train['TARGET']==1]['TARGET'].sum())

In [ ]:
# ランダムサンプリング後のデータの確認、基礎統計量
## データの確認
print("\n======= train data =======\n")
application_train.shape
application_train.columns
application_train.info()
application_train.head(5)

## 基礎統計量
print("\n======= train describe =======\n")
application_train.describe().T

# 学習データとテストデータを結合

In [ ]:
# 学習データとテストデータを縦に結合
all_df = pd.concat([application_train,application_test])

# 結合データを確認
print("\n======= all data =======\n")
all_df.shape
all_df.columns
all_df.info()
all_df.head(5)

## 基礎統計量
print("\n======= all describe =======\n")
all_df.describe().T

In [ ]:
# 後程使うtrain, testのIDを保存
## train, testよりSK_ID_CURR列の要素をそれぞれリスト化
train_list = application_train['SK_ID_CURR'].tolist()
test_list = application_test['SK_ID_CURR'].tolist()

# メモリ節約のためtrain, testのメモリを解放
del application_train, application_test

# 欠損状況の確認と処理

In [ ]:
# 欠損値処理用の関数
def missing_handle(df):
    
    # 目的変数を持つall_dfのみ別処理
    if "TARGET" in list(df.columns):
        for col in df.columns:
            if col != 'TARGET':
                if df[col].dtype=="O":
                    df[col] = df[col].fillna("None")
                else:
                    df[col] = df[col].fillna(-9999)
                    
    # all_df以外はこちらで処理
    else:
        for col in df.columns:
            if df[col].dtype=="O":
                df[col] = df[col].fillna("None")
            else:
                df[col] = df[col].fillna(-9999)

In [ ]:
# 欠損値の処理
## 欠損状況の確認
## train/testデータの確認
print("\n======= 欠損状況確認 =======\n")
all_df.isnull().sum()

print("\n======= 欠損割合確認 =======\n")
all_df.isnull().sum()/len(all_df)

## train/testデータの処理
## object型の時はNone, それ以外の場合は-9999で欠損値処理
missing_handle(all_df)
        
# 再度欠損状況の確認
print("\n======= 欠損処理後の状況を確認 =======\n")
all_df.isnull().sum()

In [ ]:
# 欠損値の処理
## 欠損状況の確認
## credit_card_balanceデータの確認
print("\n======= 欠損状況確認 =======\n")
credit_card_balance.isnull().sum()

print("\n======= 欠損割合確認 =======\n")
credit_card_balance.isnull().sum()/len(credit_card_balance)

## credit_card_balanceデータの処理
## object型の時はNone, それ以外の場合は-9999で欠損値処理
missing_handle(credit_card_balance)
        
# 再度欠損状況の確認
print("\n======= 欠損処理後の状況を確認 =======\n")
credit_card_balance.isnull().sum()

In [ ]:
# 欠損値の処理
## 欠損状況の確認# 欠損状況の確認
## POS_CASH_balanceデータの確認
print("\n======= 欠損状況確認 =======\n")
credit_card_balance.isnull().sum()

print("\n======= 欠損割合確認 =======\n")
POS_CASH_balance.isnull().sum()/len(POS_CASH_balance)

## POS_CASH_balanceデータの処理
## object型の時はNone, それ以外の場合は-9999で欠損値処理
missing_handle(POS_CASH_balance)
        
# 再度欠損状況の確認
print("\n======= 欠損処理後の状況を確認 =======\n")
POS_CASH_balance.isnull().sum()

In [ ]:
# 欠損値の処理
## 欠損状況の確認
## previous_applicationデータの確認
print("\n======= 欠損状況確認 =======\n")
previous_application.isnull().sum()

print("\n======= 欠損割合確認 =======\n")
previous_application.isnull().sum()/len(previous_application)

## previous_applicationデータの処理
## object型の時はNone, それ以外の場合は-9999で欠損値処理
missing_handle(previous_application)
        
# 再度欠損状況の確認
print("\n======= 欠損処理後の状況を確認 =======\n")
previous_application.isnull().sum()

In [ ]:
# 欠損値の処理
# 欠損状況の確認
## bureauデータの確認
print("\n======= 欠損状況確認 =======\n")
bureau.isnull().sum()

print("\n======= 欠損割合確認 =======\n")
bureau.isnull().sum()/len(bureau)

## bureauデータの処理
## object型の時はNone, それ以外の場合は-9999で欠損値処理
missing_handle(bureau)
        
# 再度欠損状況の確認
print("\n======= 欠損処理後の状況を確認 =======\n")
bureau.isnull().sum()

# 特徴量作成

## 特徴量の作成

In [ ]:
# 他社の過去ローンの返済金額割合
## （返済金額）/（ローン総額）
### ローン総額が0以下となる場合返済金額割合を0とする
bureau['REPAYMENT_RATE'] = bureau['AMT_CREDIT_SUM_DEBT'].where(bureau['AMT_CREDIT_SUM']>0, 0) / bureau['AMT_CREDIT_SUM'].where(bureau['AMT_CREDIT_SUM']>0, 1)
bureau['OVERDUE_RATE'] = bureau['AMT_CREDIT_SUM_OVERDUE'].where(bureau['AMT_CREDIT_SUM']>0, 0) / bureau['AMT_CREDIT_SUM'].where(bureau['AMT_CREDIT_SUM']>0, 1)

# クレジットカードのローン残金割合
## （返済金額）/（ローン総額）
### ローン総額が0以下の場合残金割合を0とする
credit_card_balance['CREDIT_REPAYMENT_RATE'] = credit_card_balance['AMT_RECIVABLE'].where(credit_card_balance['AMT_TOTAL_RECEIVABLE']>0, 0) / credit_card_balance['AMT_TOTAL_RECEIVABLE'].where(credit_card_balance['AMT_TOTAL_RECEIVABLE']>0, 1)

In [ ]:
# 各データフレームの必要カラムのみを抽出
## データフレーム毎の必要カラム一覧
### application_{train/test}.csv
application_list = [
    "AMT_INCOME_TOTAL",
    "DAYS_BIRTH",
    "OBS_30_CNT_SOCIAL_CIRCLE",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "NAME_TYPE_SUITE",
    "AMT_GOODS_PRICE",
]

### credit_card_balance.csv
credit_balance_list = [
    "AMT_CREDIT_LIMIT_ACTUAL",
    # 特徴量追加用
    "CREDIT_REPAYMENT_RATE",
    "AMT_TOTAL_RECEIVABLE",
]

### POS_CASH_balance.csv
pos_balance_list = [
    "SK_DPD",
]

### previous_application.csv
previous_application_list = [
    "AMT_CREDIT",
    "RATE_INTEREST_PRIMARY",
    "RATE_INTEREST_PRIVILEGED",
    "AMT_ANNUITY",
    "NAME_CONTRACT_STATUS",
    "NAME_CLIENT_TYPE",
    "NAME_TYPE_SUITE",
]

### bureau.csv
bureau_list = [
    "AMT_CREDIT_SUM",
    "AMT_ANNUITY",
    "CREDIT_ACTIVE",
    "REPAYMENT_RATE",
    "CREDIT_DAY_OVERDUE",
    "CNT_CREDIT_PROLONG",
]

## 必要カラムのみ抽出
### bureau, previous_applicationに関してはall_dfと同名のカラム名をリネーム処理
all_df = all_df[['SK_ID_CURR','TARGET'] + application_list]
bureau = bureau[['SK_ID_CURR'] + bureau_list].rename(columns={'AMT_ANNUITY': 'AMT_ANNUITY_BUREAU'}) # 同名のカラム名をリネーム処理
previous_application = previous_application[['SK_ID_CURR'] + previous_application_list].rename(columns={'AMT_ANNUITY': 'AMT_ANNUITY_PREVIOUS'}) # 同名のカラム名をリネーム処理
POS_CASH_balance = POS_CASH_balance[['SK_ID_CURR'] + pos_balance_list]
credit_card_balance = credit_card_balance[['SK_ID_CURR'] + credit_balance_list]

## カテゴリ変数をダミー変数化

In [ ]:
# カテゴリ変数をダミー変数化
## all_dfのNAME_TYPE_SUITEを変換
all_df = pd.get_dummies(all_df, columns=['NAME_TYPE_SUITE']).rename(columns={'NAME_TYPE_SUITE_Spouse, partner': 'NAME_TYPE_SUITE_Spouse_partner'})

## bureauのCREDIT_ACTIVEを変換
bureau = pd.get_dummies(bureau, columns=['CREDIT_ACTIVE'])

## previous_applicationのNAME_CONTRACT_STATUS, NAME_CLIENT_TYPEを変換
previous_application = pd.get_dummies(previous_application, columns=['NAME_CONTRACT_STATUS', 'NAME_CLIENT_TYPE', 'NAME_TYPE_SUITE']).rename(columns={'NAME_TYPE_SUITE_Spouse, partner': 'NAME_TYPE_SUITE_Spouse_partner'})

In [ ]:
# 必要データ抽出後のデータ確認
## train/test データ確認
print("\n======= train/test data =======\n")
all_df.shape
all_df.columns
all_df.info()
all_df.head(5)

## bureau.csv データ確認
print("\n======= bureau data =======\n")
bureau.shape
bureau.columns
bureau.info()
bureau.head(5)

## previous_application.csv データ確認
print("\n======= previous_application data =======\n")
previous_application.shape
previous_application.columns
previous_application.info()
previous_application.head(5)

## POS_CASH_balance.csv データ確認
print("\n======= POS_CASH_balance data =======\n")
POS_CASH_balance.shape
POS_CASH_balance.columns
POS_CASH_balance.info()
POS_CASH_balance.head(5)

## credit_card_balance.csv データ確認
print("\n======= credit_card_balance data =======\n")
credit_card_balance.shape
credit_card_balance.columns
credit_card_balance.info()
credit_card_balance.head(5)

## ID毎にカラムを集計
all_df以外のデータはSK_ID_CURRに紐づくデータが複数存在する可能性があるため、最大値や平均といった値を取り分析に利用する

In [ ]:
# 各データフレームで集計する項目を辞書化
## bureauの辞書
aggregations_bureau = {
    "AMT_CREDIT_SUM":['max', 'mean'],
    "AMT_ANNUITY_BUREAU":['max', 'mean'],
    "REPAYMENT_RATE":['min', 'max', 'mean'],
    "CREDIT_DAY_OVERDUE":['max', 'mean'],
    "CNT_CREDIT_PROLONG":['max', 'mean'],
    "CREDIT_ACTIVE_Active":['count'],
    "CREDIT_ACTIVE_Bad debt":['count'],
    "CREDIT_ACTIVE_Closed":['count'],
    "CREDIT_ACTIVE_Sold":['count'],
}

## previous_applicationの辞書
aggregations_previous = {
    "AMT_CREDIT":['max', 'mean'],
    "RATE_INTEREST_PRIMARY":['min', 'max', 'mean'],
    "RATE_INTEREST_PRIVILEGED":['min', 'max', 'mean'],
    "AMT_ANNUITY_PREVIOUS":['max', 'mean'],
    "NAME_CONTRACT_STATUS_Approved":['count'],
    "NAME_CONTRACT_STATUS_Refused":['min', 'count'],
    "NAME_CLIENT_TYPE_New":['min'],
    "NAME_CLIENT_TYPE_Refreshed":['min', 'count'],
    "NAME_CLIENT_TYPE_Repeater":['count'],
    "NAME_TYPE_SUITE_Children":['count', 'max'],
    "NAME_TYPE_SUITE_Family":['count', 'max'],
    "NAME_TYPE_SUITE_Group of people":['count', 'max'],
    "NAME_TYPE_SUITE_Other_A":['count', 'max'],
    "NAME_TYPE_SUITE_Other_B":['count', 'max'],
    "NAME_TYPE_SUITE_Spouse_partner":['count', 'max'],
    "NAME_TYPE_SUITE_Unaccompanied":['count', 'max'],
}

## POS_CASHの辞書
aggregations_pos = {
    "SK_DPD":['max', 'mean'],
}

## credit_cardの辞書
aggregations_credit_card = {
    "AMT_CREDIT_LIMIT_ACTUAL":['min', 'max', 'mean'],
    "CREDIT_REPAYMENT_RATE":['min', 'max', 'mean'],
    "AMT_TOTAL_RECEIVABLE":['max', 'mean'],
}

In [ ]:
# 各データフレームで集計
## bureau
bureau_agg = bureau.groupby('SK_ID_CURR').agg(aggregations_bureau)
## カラム名が二重になるため修正
bureau_agg.columns = pd.Index(['bureau_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
## SK_ID_CURRがインデックスとなっているので列に追加
bureau_agg = bureau_agg.reset_index()

## previous_application
previous_agg = previous_application.groupby('SK_ID_CURR').agg(aggregations_previous)
## カラム名が二重になるため修正
previous_agg.columns = pd.Index(['previous_' + e[0] + "_" + e[1].upper() for e in previous_agg.columns.tolist()])
## SK_ID_CURRがインデックスとなっているので列に追加
previous_agg = previous_agg.reset_index()

## POS_CASH_balance
pos_agg = POS_CASH_balance.groupby('SK_ID_CURR').agg(aggregations_pos)
## カラム名が二重になるため修正
pos_agg.columns = pd.Index(['pos_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
## SK_ID_CURRがインデックスとなっているので列に追加
pos_agg = pos_agg.reset_index()

## credit_card_balance
credit_agg = credit_card_balance.groupby('SK_ID_CURR').agg(aggregations_credit_card)
## カラム名が二重になるため修正
credit_agg.columns = pd.Index(['credit_' + e[0] + "_" + e[1].upper() for e in credit_agg.columns.tolist()])
## SK_ID_CURRがインデックスとなっているので列に追加
credit_agg = credit_agg.reset_index()

In [ ]:
# 集計後のデータ確認
## bureau.csv データ確認
print("\n======= bureau data =======\n")
bureau_agg.shape
bureau_agg.columns
bureau_agg.info()
bureau_agg.head(5)

## previous_application.csv データ確認
print("\n======= previous_application data =======\n")
previous_agg.shape
previous_agg.columns
previous_agg.info()
previous_agg.head(5)

## POS_CASH_balance.csv データ確認
print("\n======= POS_CASH_balance data =======\n")
pos_agg.shape
pos_agg.columns
pos_agg.info()
pos_agg.head(5)

## credit_card_balance.csv データ確認
print("\n======= credit_card_balance data =======\n")
credit_agg.shape
credit_agg.columns
credit_agg.info()
credit_agg.head(5)

In [ ]:
# 基礎統計量を確認
## bureau.csv 基礎統計量確認
print("\n======= bureau data =======\n")
bureau_agg.describe().T

## previous_application.csv 基礎統計量確認
print("\n======= previous_application data =======\n")
previous_agg.describe().T

## POS_CASH_balance.csv 基礎統計量確認
print("\n======= POS_CASH_balance data =======\n")
pos_agg.describe().T

## credit_card_balance.csv 基礎統計量確認
print("\n======= credit_card_balance data =======\n")
credit_agg.describe().T

## all_dfとbureauを列方向へ結合
列方向の結合はデータフレーム一つ一つを順にmergeを用いて結合させる必要がある

In [ ]:
# PKが想定通りに設定されているかを確認
## 結合前のSK_ID_CURRの個数を確認
print(len(all_df['SK_ID_CURR']))

# train/testデータに目的変数を左外部結合
## 左外部結合の場合mergeを利用し一つずつ結合させなければならない
all_df = (
    all_df
    .merge(bureau_agg, how='left', on='SK_ID_CURR')
    .merge(previous_agg, how='left', on='SK_ID_CURR')
    .merge(pos_agg, how='left', on='SK_ID_CURR')
    .merge(credit_agg, how='left', on='SK_ID_CURR')
)

#結合後のSK_ID_CURRの個数を確認
print(len(all_df['SK_ID_CURR']))

In [ ]:
# 改めて欠損値の処理
## 欠損状況の確認
## train/testデータの確認
print("\n======= 欠損状況確認 =======\n")
all_df.isnull().sum()

print("\n======= 欠損割合確認 =======\n")
all_df.isnull().sum()/len(all_df)

## train/testデータの処理
## object型の時はNone, それ以外の場合は-9999で欠損値処理
missing_handle(all_df)
        
# 再度欠損状況の確認
print("\n======= 欠損処理後の状況を確認 =======\n")
all_df.isnull().sum()

In [ ]:
# 結合データを確認
print("\n======= all data =======\n")
all_df.shape
all_df.columns
all_df.info()
all_df.head(5)

## 基礎統計量
print("\n======= all describe =======\n")
all_df.describe().T

## 強相関の除去

In [ ]:
corr_mat = all_df.corr(method='pearson')

sns.set(rc = {'figure.figsize':(18,11)})
sns.heatmap(corr_mat,
            vmin=-1.0,
            vmax=1.0,
            center=0,
            annot=True, # True:格子の中に値を表示
            fmt='.1f',
            xticklabels=corr_mat.columns.values,
            yticklabels=corr_mat.columns.values
           )
plt.show()

In [ ]:
# 強相関変数の片方を除外
dtype_list = all_df.dtypes
numerical_variables = dtype_list.index
corrmat = all_df[numerical_variables].corr()

drop_tar = []
THRESHOLD = 0.8 # 閾値
for i, col in enumerate(corrmat.columns):
    drop_tar_flg = 0
    if i == 0:
        pass
    else:
        for j in range(i-1):
            if abs(corrmat.iloc[i,j]) > THRESHOLD:
                print(f"drop_tar >> col:{col}, index:{corrmat.index[j]}, corr_coef:{corrmat.iloc[i,j]}")
                drop_tar_flg = 1
    if drop_tar_flg == 1:
        drop_tar += [col]

print(f"""
----------------------------------------
除外対象 :
{drop_tar}
----------------------------------------
len(drop_tar) : {len(drop_tar)}
len(np.unique(drop_tar)) : {len(np.unique(drop_tar))}
""")


all_df.drop(drop_tar ,axis=1, inplace=True)

print(f"""
除外結果カラム一覧 :
{all_df.columns}
""")

# モデリング
## 学習用データと予測用データへ分割

In [ ]:
# 学習用と予測用にデータを分割
X_all_train = all_df[~all_df['TARGET'].isna()]
X_test = all_df[all_df['TARGET'].isna()].drop(['SK_ID_CURR', 'TARGET'], axis=1)

y_all_train = X_all_train['TARGET']
X_all_train = X_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)

# 各データの形状を確認
print("X_all_train : " + str(X_all_train.shape))
print("X_test : " + str(X_test.shape))
print("y_all_train : " + str(y_all_train.shape))

## 特徴量を正規化（標準化）

In [ ]:
scaler = StandardScaler()
# 学習データのみを使用してパラメータを計算
scaler.fit(X_all_train)

# 学習データと予測データそれぞれを標準化
X_all_train = pd.DataFrame(scaler.transform(X_all_train), columns=X_all_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# 標準化されたデータを確認
X_all_train.head()
X_test.head()

## 学習用と検証用へデータを分割

In [ ]:
# 学習用データを学習用と検証用にさらに分割
X_train, X_valid, y_train, y_valid = train_test_split(X_all_train, y_all_train, test_size = 0.3, random_state = 42)


# 各データの形状を確認
print("X_train : " + str(X_train.shape))
print("X_valid : " + str(X_valid.shape))
print("y_train : " + str(y_train.shape))
print("y_valid : " + str(y_valid.shape))
type(X_train)

## ロジスティクス回帰

In [ ]:
# ロジスティック回帰で学習
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

#　検証用データにて予測
logistic_pred = logistic_model.predict_proba(X_valid)[:,1]
# aucプロット用データを計算
fpr, tpr, thresholds = roc_curve(y_true=y_valid, y_score=logistic_pred)

plt.plot(fpr, tpr, label='roc curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], linestyle='--', label='random')
plt.plot([0, 0, 1], [0, 1, 1], linestyle='--', label='ideal')
plt.legend()
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()

# aucスコアを計算
print('auc = ', roc_auc_score(y_true=y_valid, y_score=logistic_pred))

## L1罰則付きロジスティクス回帰

In [ ]:
# l1罰則付きロジスティクス回帰で学習
logistic_l1_model = LogisticRegression(penalty='l1', 
                                      solver='liblinear',
                                      tol=1e-6, max_iter=int(1e6),
                                      warm_start=True,
                                      intercept_scaling=10000.)
logistic_l1_model.fit(X_train, y_train)

#　検証用データにて予測
logistic_l1_pred = logistic_l1_model.predict_proba(X_valid)[:,1]
# aucプロット用データを計算
fpr, tpr, thresholds = roc_curve(y_true=y_valid, y_score=logistic_l1_pred)

plt.plot(fpr, tpr, label='roc curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], linestyle='--', label='random')
plt.plot([0, 0, 1], [0, 1, 1], linestyle='--', label='ideal')
plt.legend()
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()

# aucスコアを計算
print('auc = ', roc_auc_score(y_true=y_valid, y_score=logistic_l1_pred))

## ランダムフォレスト

In [ ]:
# ランダムフォレストで学習
forest_model = RandomForestClassifier(random_state=42)
forest_model.fit(X_train, y_train)

#　検証用データにて予測
forest_pred = forest_model.predict_proba(X_valid)[:,1]
# aucプロット用データを計算
fpr, tpr, thresholds = roc_curve(y_true=y_valid, y_score=forest_pred)

plt.plot(fpr, tpr, label='roc curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], linestyle='--', label='random')
plt.plot([0, 0, 1], [0, 1, 1], linestyle='--', label='ideal')
plt.legend()
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()

# aucスコアを計算
print('auc = ', roc_auc_score(y_true=y_valid, y_score=forest_pred))

## xgboost

In [ ]:
# xgboostで学習
# パラメータを設定
params = {
    # 二値分類問題
    "objective": "binary:logistic",
    # 評価指標
    "eval_metric": "logloss",
    "metric": "auc",
    "verbosity": 0,
    "seed": 42,
}

# XGBoostが扱うデータセット形式へ変換
xgb_train = xgb.DMatrix(X_train, label=y_train)
xgb_val = xgb.DMatrix(X_valid, label=y_valid)

# XGBoostにて学習
xgb_model = xgb.train(params,
                      xgb_train,
                      num_boost_round=100,
                      )

#　検証用データにて予測
xgb_pred = xgb_model.predict(xgb_val)
# aucプロット用データを計算
fpr, tpr, thresholds = roc_curve(y_true=y_valid, y_score=xgb_pred)

plt.plot(fpr, tpr, label='roc curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], linestyle='--', label='random')
plt.plot([0, 0, 1], [0, 1, 1], linestyle='--', label='ideal')
plt.legend()
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()

# aucスコアを計算
print('auc = ', roc_auc_score(y_true=y_valid, y_score=xgb_pred))

## LightGBM

In [ ]:
# LightGBMで学習
params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "seed": 42,
}

# LightGBMが扱うデータセット形式へ変換
train_set = lgb.Dataset(X_train, y_train)
val_set = lgb.Dataset(X_valid, y_valid)

# LightGBMにて学習
lgb_model = lgb.train(params,
                      train_set=train_set,
                      valid_sets=[train_set, val_set],
                     )

#　検証用データにて予測
lgb_pred = lgb_model.predict(X_valid)
# aucプロット用データを計算
fpr, tpr, thresholds = roc_curve(y_true=y_valid, y_score=lgb_pred)

plt.plot(fpr, tpr, label='roc curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], linestyle='--', label='random')
plt.plot([0, 0, 1], [0, 1, 1], linestyle='--', label='ideal')
plt.legend()
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()

# aucスコアを計算
print('auc = ', roc_auc_score(y_true=y_valid, y_score=lgb_pred))

## 各スコア比較

In [ ]:
print("\n------------------ロジスティクス回帰------------------\n")
print('auc = ', roc_auc_score(y_true=y_valid, y_score=logistic_pred))
print("\n------------------L1罰則付きロジスティクス回帰------------------\n")
print('auc = ', roc_auc_score(y_true=y_valid, y_score=logistic_l1_pred))
print("\n------------------ランダムフォレスト------------------\n")
print('auc = ', roc_auc_score(y_true=y_valid, y_score=forest_pred))
print("\n------------------XGBoost------------------\n")
print('auc = ', roc_auc_score(y_true=y_valid, y_score=xgb_pred))
print("\n------------------LightGBM(パラメータチューニングなし)------------------\n")
print('auc = ', roc_auc_score(y_true=y_valid, y_score=lgb_pred))

## Feature importanceとSHAP

In [ ]:
# 変数重要度
importance = pd.DataFrame(lgb_model.feature_importance(importance_type='gain'), index=X_all_train.columns, columns=['importance'])
display(importance)

lgb.plot_importance(lgb_model, figsize=(8,4), max_num_features=5, importance_type='gain')

### SHAP

In [ ]:
# 下準備
# データが各クラスに分類される確率を計算
y_pred_test = lgb_model.predict(X_valid)
y_pred_train = lgb_model.predict(X_train)

shap.initjs()
explainer = shap.TreeExplainer(lgb_model)

# trainデータのSHAP値
shap_values_train = explainer.shap_values(X_train)

# testデータのSHAP値
shap_values_test = explainer.shap_values(X_valid)

In [ ]:
plt.figure()
plt.hist([y_pred_train, y_pred_test], bins=10, density=True, color=["blue", "orange"], label=["train", "test"])
plt.legend()
plt.grid()
plt.title("Score Distribution")
plt.show()

In [ ]:
# 要約プロット
# shap値ベースでの特徴量の評価
# 通常の変数重要度とは異なることに注意
# どちらが良く特徴量を評価するか要確認
plt.figure()
shap.summary_plot(shap_values=shap_values_train, features=X_train, feature_names=X_train.columns)

In [ ]:
# 全サンプルについて、各特徴量のSHAP値の分布を可視化したもの
# 上記の棒グラフ同様、予測に対する寄与度が大きい順にソートされて表示される
# 色は特徴量の値の大小を表している(赤が大きくて青が小さい)
plt.figure()

shap.summary_plot(shap_values_train,
                  X_train,
                  show=False,
                 )

## 学習済みモデルで予測

In [ ]:
xgb_test = xgb.DMatrix(X_test)

# 各モデルで予測し、結果を格納
test_pred_logi = all_df[all_df['TARGET'].isna()]["SK_ID_CURR"]
test_pred_logi['TARGET'] = logistic_model.predict(X_test)

test_pred_l1 = all_df[all_df['TARGET'].isna()]["SK_ID_CURR"]
test_pred_l1['TARGET'] = logistic_l1_model.predict(X_test)

test_pred_forest = all_df[all_df['TARGET'].isna()]["SK_ID_CURR"]
test_pred_forest['TARGET'] = forest_model.predict(X_test)

test_pred_xgb = all_df[all_df['TARGET'].isna()]["SK_ID_CURR"]
test_pred_xgb['TARGET'] = xgb_model.predict(xgb_test)

test_pred_lgb = all_df[all_df['TARGET'].isna()]["SK_ID_CURR"]
test_pred_lgb['TARGET'] = lgb_model.predict(X_test)

## ハイパーパラメータチューニング
ハイパーパラメータチューニングにあたり、クロスバリデーションとハイパーパラメータチューニングの概要に関する簡単な説明が乗っているサイト
- [クロスバリデーション・ハイパーパラメータチューニングとは](https://free.kikagaku.ai/tutorial/basic_of_machine_learning/learn/machine_learning_hyperparameters)  

ハイパーパラメータについての情報を下表にまとめる。なお、主に下記2点を参考にした。
- [XGBClassifier API](https://xgboost.readthedocs.io/en/latest/python/python_api.html)
- [xgboost API(model)](https://xgboost.readthedocs.io/en/latest/parameter.html)

|パラメータ|詳細|定義域|デフォルト値|取る頻度が高い値|
|:-:|:-|:-:|:-:|:-:|
|n_estimators|勾配ブーストされた木の数.ブーストラウンドの数に相当.|[0,∞]|100|-|
|use_label_encoder|scikit-learn のラベルエンコーダーを使ってラベルをエンコード.このパラメータはFalseに設定することを推奨.|-|不明|-|
|max_depth|最大の木の深さ.値が大きいと木が複雑になり過学習が発生しやすくなる.|[0,∞]|6|3~10|
|learning_rate|ブースト学習率(xgbにおける"eta").値が小さいと学習が進みにくい（時間がかかる）一方で過学習に陥りにくい.|[0,1]|0.3|0.01~0.2|
|verbosity|冗長性の度合い.|0(silent)~3(debug)|1|-|
|objective|学習タスクと,それに対応する学習目的,または使用するカスタム目的関数を指定.|[公式API参照](https://xgboost.readthedocs.io/en/latest/parameter.html)|'reg:linear'|-|
|booster|使用するブースターを指定.|[公式API参照](https://xgboost.readthedocs.io/en/latest/parameter.html)|'gbtree'|-|
|tree_method|使用するツリー法を指定.このパラメータがデフォルトに設定されている場合, XGBoostは利用可能な最も保守的なオプションを選択.|[公式API参照](https://xgboost.readthedocs.io/en/latest/parameter.html)|'auto'|-|
|n_jobs|xgboostの実行に使用される並列スレッドの数.-1を指定すると全コアで並列計算を行う.|-|1|-|
|gamma|木の複雑さ）損失関数の減少がこの値を超える時にのみ分割を進める.|[0,∞]|0|-|
|min_child_weight|子に必要なインスタンスウェイト（ヘシアン）の最小値の合計.分割中にノードの重みがこれを超えた時点で分割をやめる.値が大きいと木がシンプルになり,過学習しにくくなる.|[0,∞]|1|-|
|max_delta_step|各決定木の重みの推定に制約をかける.0の場合は制約なしとなり、整数値を設定するとモデルをより保守的にする.不均衡データで用いられる.|[0,∞]|0|1~10|
|subsample|トレーニングインスタンスのサブサンプル比率.各ステップの木を作る際に用いるレコードの数をサンプリングする割合.サンプリングすると過学習を防げる.|(0,1]|1|0.5~1|
|colsample_bytree|各ツリーを構築する際の列のサブサンプル比率.各ステップの木を作る際に用いる列の数をサンプリングする割合。サンプリングすると過学習を防げる.|(0,1]|1|0.5~1|
|colsample_bylevel|各レベルの列のサブサンプル比率.|(0,1]|1|0.5~1|
|colsample_bynode|各分割のための列のサブサンプル比率.|(0,1]|1|0.5~1|
|reg_alpha|重みに対するL1正則化項（xgbのアルファ）.大きくするとモデルが保守的になる.|-|0|-|
|reg_lambda|重みに対するL2正則化項（xgbのlambda）.大きくするとモデルが保守的になる.|-|1|-|
|scale_pos_weight|正負の重みのバランスをとる.|[0,∞]|1|sum(negative instances) / sum(positive instances)|
|base_score|全インスタンスの初期予測スコア,グローバルバイアス.反復回数が十分な場合,この値を変更しても効果はない.|-|0.5|-|
|random_state|乱数シード.|-|不明|-|

### クロスバリデーションの設定

In [ ]:
# クロスバリデーション分割データ数
N_SPLITS = 5

# データ分割
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=0)

### Optunaに関する設定

In [ ]:
# optuna 目的関数(LightGBM)
def objective(trial):

    # パラメータの設定
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "seed": 42,
        # L2 正則化
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-3, 1e3),
        # L1 正則化
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-3, 1e3),
        # 学習に関する制限
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0),
        "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
        # 木の制限
        "max_depth": trial.suggest_int("max_depth", 3, 7),
        "min_child_weight": trial.suggest_uniform("min_child_weight", 0.5, 5),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 2.0),
    }
    
    # 学習記録用の入れ物
    scores  = 0.0 # validation用
    
    # 学習データの数だけの数列（0行から最終行まで連番）
    row_no_list = list(range(len(y_train)))
    
    for trn_idx, val_idx in skf.split(row_no_list, y_train):
        x_trn = X_train.iloc[trn_idx, :]
        y_trn = pd.Series(y_train).iloc[trn_idx]
        x_val = X_train.iloc[val_idx, :]
        y_val = pd.Series(y_train).iloc[val_idx]
        
        lgb_train = lgb.Dataset(x_trn, y_trn)
        lgb_eval = lgb.Dataset(x_val, y_val)
        
        model = lgb.train(params=params,
                          train_set=lgb_train,
                          valid_sets=[lgb_train, lgb_eval],
                          num_boost_round=1000, # 100 round まで実行
                          early_stopping_rounds=100,
                          verbose_eval=100)
        
        # modelから予測値を出す
        y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
        
        # roc-auc
        fpr, tpr, thresholds = metrics.roc_curve(y_valid, y_pred)
        auc = metrics.auc(fpr, tpr)
        
        # 各バリデーションスコアの平均
        scores += auc / N_SPLITS
    
    return scores

### パラメータチューニングおよび学習

In [ ]:
study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=500)

In [ ]:
# 最良パラメータでの学習用データをLightGBM用に変換
lgb_train_all = lgb.Dataset(X_train, y_train)
lgb_test_all = lgb.Dataset(X_valid, y_valid, reference=lgb_train_all)

In [ ]:
# 最良のパラメータを代入
params = study.best_params
params["objective"] = "binary"
params["metric"] = "binary_logloss"

# パフォーマンス監視用（学習曲線）
evals_result = {}

optuna_model = lgb.train(params,
                      lgb_train_all,
                      valid_sets = [lgb_train_all, lgb_test_all],
                      valid_names = ["train", "test"],
                      num_boost_round=1000,
                      early_stopping_rounds=100,
                      evals_result=evals_result,
                      verbose_eval=100)

In [ ]:
# 最良parameterの結果を出力
print(f"""
best results--------------------
best parameter : {study.best_params}
best value : {study.best_value}
best trial : {study.best_trial}
""")

In [ ]:
#　検証用データにて予測
optuna_pred = optuna_model.predict(X_valid)
# aucプロット用データを計算
fpr, tpr, thresholds = roc_curve(y_true=y_valid, y_score=optuna_pred)

plt.plot(fpr, tpr, label='roc curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], linestyle='--', label='random')
plt.plot([0, 0, 1], [0, 1, 1], linestyle='--', label='ideal')
plt.legend()
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()

# aucスコアを計算
print('auc = ', roc_auc_score(y_true=y_valid, y_score=optuna_pred))

## 各手法のスコアを比較

In [ ]:
print("\n------------------ロジスティクス回帰------------------\n")
print('auc = ', roc_auc_score(y_true=y_valid, y_score=logistic_pred))
print("\n------------------L1罰則付きロジスティクス回帰------------------\n")
print('auc = ', roc_auc_score(y_true=y_valid, y_score=logistic_l1_pred))
print("\n------------------ランダムフォレスト------------------\n")
print('auc = ', roc_auc_score(y_true=y_valid, y_score=forest_pred))
print("\n------------------XGBoost------------------\n")
print('auc = ', roc_auc_score(y_true=y_valid, y_score=xgb_pred))
print("\n------------------LightGBM(パラメータチューニングなし)------------------\n")
print('auc = ', roc_auc_score(y_true=y_valid, y_score=lgb_pred))
print("\n------------------LightGBM+optuna(パラメータチューニングあり)------------------\n")
print('auc = ', roc_auc_score(y_true=y_valid, y_score=optuna_pred))

## Feature importanceとSHAP
SHAPの可視化に関しては以下のサイトを参考
- https://qiita.com/perico_v1/items/fbbb18681ecc362a4f9e  
- https://qiita.com/shin_mura/items/cde01198552eda9146b7

### Feature importance

In [ ]:
# 変数重要度
importance_optuna = pd.DataFrame(optuna_model.feature_importance(importance_type='gain'), index=X_all_train.columns, columns=['importance'])
display(importance)

lgb.plot_importance(optuna_model, figsize=(8,4), max_num_features=5, importance_type='gain')

### Shap

In [ ]:
# 下準備
# データが各クラスに分類される確率を計算
y_pred_test = optuna_model.predict(X_valid)
y_pred_train = optuna_model.predict(X_train)

shap.initjs()
explainer = shap.TreeExplainer(optuna_model)

# trainデータのSHAP値
shap_values_train = explainer.shap_values(X_train)

# testデータのSHAP値
shap_values_test = explainer.shap_values(X_valid)

### スコアの分布を確認

In [ ]:
plt.figure()
plt.hist([y_pred_train, y_pred_test], bins=10, density=True, color=["blue", "orange"], label=["train", "test"])
plt.legend()
plt.grid()
plt.title("Score Distribution")
plt.show()

### SHAPの可視化

In [ ]:
# 要約プロット
# shap値ベースでの特徴量の評価
# 通常の変数重要度とは異なることに注意
# どちらが良く特徴量を評価するか要確認
plt.figure()
shap.summary_plot(shap_values=shap_values_train, features=X_train, feature_names=X_train.columns)

In [ ]:
# 全サンプルについて、各特徴量のSHAP値の分布を可視化したもの
# 上記の棒グラフ同様、予測に対する寄与度が大きい順にソートされて表示される
# 色は特徴量の値の大小を表している(赤が大きくて青が小さい)
plt.figure()

shap.summary_plot(shap_values_train,
                  X_train,
                  show=False,
#                   max_display=10,
                 )